In [56]:
import spacy
import sys
import pickle
import numpy as np
from collections import defaultdict
nlp = spacy.load('en')

In [57]:
v = '2'

path = 'mimic/' + v + '/MIMIC_FILTERED_DSUMS'

labels = []
texts = []
for line in open(path, encoding='utf-8'):
    line = line.strip().split('|')
    labels.append(set([label for label in line[0].split(',')]))
    texts.append(line[1])

x = []
y = []

all_words = set()
all_labels = defaultdict(set)

for i, doc in enumerate(nlp.pipe(texts, n_threads=40)):
    for label in labels[i]:
        
        label = label.split('.')
        if len(label) == 2:
            all_labels[label[0]].add(label[1])
        else:
            all_labels[label[0]].add('-1')
    x_doc = []
    for sent in doc.sents:
        x_sent = []
        for word in sent:
            word = str(word)
            all_words.add(word)
            x_sent.append(word)
        x_doc.append(x_sent)
    
    x.append(x_doc)
    y.append(labels[i])

In [58]:
all_cats = list(all_labels.keys())
all_words = list(all_words)
l_to_ints = {}
for i, cat in enumerate(all_labels.keys()):
    for j, label in enumerate(all_labels[cat]):
        if label == '-1':
            l_to_ints[cat] = (i, j)
        else:
            l_to_ints[cat+'.'+label] = (i, j)

w_to_int = {word: i for i, word in enumerate(all_words)}

In [59]:
y = [[l_to_ints[l] for l in doc]for doc in y]
x = [[[w_to_int[w] for w in sent] for sent in doc]for doc in x]

In [60]:
with open('mimic/'+v+'unrolled.p',mode='wb') as fp:
    data = (x, y, all_words, all_labels, all_cats)
    pickle.dump(data, fp)